In [1]:
import autograd.numpy as np
from autograd import grad
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#### Load the MNIST dataset

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(int)

D:\ANACONDA\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Scale the feature values
scaler  = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [8]:
# Perform one-hot encoding on the target variable
encoder = OneHotEncoder(sparse=False)

y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))


D:\ANACONDA\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# Define the softmax function
def softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

In [11]:
# Define the cross-entropy loss function
def cross_entropy_loss(theta, X, y):
    logits        = np.dot(X, theta)
    probabilities = softmax(logits)
    
    loss          = -np.mean(np.sum(y * np.log(probabilities), axis=1))
    return loss

In [12]:
# Initialize the model parameters
np.random.seed(42)

num_features = X_train.shape[1]
num_classes  = len(np.unique(y_train))

theta_initial = np.random.randn(num_features, num_classes)

In [13]:
# Compute the gradients using autograd
gradient = grad(cross_entropy_loss)

In [14]:
# Perform gradient descent optimization
learning_rate  = 0.1
num_iterations = 100

In [15]:
theta = theta_initial

In [16]:
for i in range(num_iterations):
    # Compute gradients
    gradients = gradient(theta, X_train, y_train_encoded)

    # Update parameters
    theta -= learning_rate * gradients

D:\ANACONDA\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
D:\ANACONDA\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: invalid value encountered in multiply
  return f_raw(*args, **kwargs)
D:\ANACONDA\lib\site-packages\autograd\numpy\numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
D:\ANACONDA\lib\site-packages\autograd\numpy\numpy_vjps.py:78: RuntimeWarning: overflow encountered in divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
D:\ANACONDA\lib\site-packages\autograd\numpy\numpy_vjps.py:78: RuntimeWarning: invalid value encountered in divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
D:\ANACONDA\lib\site-packages\autograd\numpy\numpy_vjps.py:53: RuntimeWarning: invalid value encountered in multiply
  lambda ans, x, y : unbroadcast_f(y, lambda g: - g * x / y**2))


In [17]:
# Evaluate the model on the test set
logits_test = np.dot(X_test, theta)

probabilities_test = softmax(logits_test)
y_pred             = np.argmax(probabilities_test, axis=1)

accuracy = np.mean(y_pred == y_test)

print("Accuracy:", accuracy)

Accuracy: 0.09592857142857143
